In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 3 Layer ReLU Feedforward Network
# Output: zonotope of dimension 2, order 2 (6 total parameters)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 16)  # 5*5 from image dimension
        self.fc2 = nn.Linear(16, 10)
        self.fc3 = nn.Linear(10, 6)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=6, bias=True)
)


In [4]:
input = torch.randn(1,4)
out = net(input)
print(out)

tensor([[-0.2830, -0.0161, -0.0738, -0.3219,  0.0284,  0.1668]],
       grad_fn=<AddmmBackward>)
